In [ ]:
import pandas as pd
import os
from config import username, password
from sqlalchemy import create_engine

# Scraping with Pandas

We can use the `read_html` function in Pandas to automatically scrape any tabular data from a page.

In [ ]:
url = 'https://animalcrossing.fandom.com/wiki/Bugs_(New_Horizons)'

In [ ]:
# Reading url table into a df
tables = pd.read_html(url)
tables[3]

## Data Transformation

In [ ]:
# storing the third table from the url in a df
df = tables[3]

# deleting the image column
del df['Image']

# displaying the df
df

In [ ]:
# Replacing check marks with True and minus signs with False
df["Jan"] = df["Jan"].replace("✓", True).replace("-", False)
df["Feb"] = df["Feb"].replace("✓", True).replace("-", False)
df["Mar"] = df["Mar"].replace("✓", True).replace("-", False)
df["Apr"] = df["Apr"].replace("✓", True).replace("-", False)
df["May"] = df["May"].replace("✓", True).replace("-", False)
df["Jun"] = df["Jun"].replace("✓", True).replace("-", False)
df["Jul"] = df["Jul"].replace("✓", True).replace("-", False)
df["Aug"] = df["Aug"].replace("✓", True).replace("-", False)
df["Sep"] = df["Sep"].replace("✓", True).replace("-", False)
df["Oct"] = df["Oct"].replace("✓", True).replace("-", False)
df["Nov"] = df["Nov"].replace("✓", True).replace("-", False)
df["Dec"] = df["Dec"].replace("✓", True).replace("-", False)

# Capitalizing the name column
df["Name"] = df["Name"].str.title()

#Remove apostophes from bug names
df["Name"] = df["Name"].replace("Rajah Brooke'S Birdwing","Rajah Brookes Birdwing")
df["Name"] = df["Name"].replace("Queen Alexandra'S Birdwing","Queen Alexandras Birdwing")



# Displaying df
df
    

## Bug ID DF


In [ ]:
# Reading id csv 
filepath = os.path.join("..", "Resources", "ids.csv")

# Reading csv into a df
id_df = pd.read_csv(filepath)

# Filtering id df to only contain Bugs 
bug_id = id_df.loc[id_df['Type']=='Bug']

## Merge ID and Bug DF

In [ ]:
# Merging bug id's and the original df
bug_id_merge = df.merge(bug_id[["ID","Type","Name"]],on='Name',how='left')

#Set all columns to lowercase
bug_id_merge.columns = bug_id_merge.columns.str.lower()

# Setting index to the id
bug_id_merge = bug_id_merge.set_index("id")

#Reorder columns
bug_id_merge = bug_id_merge[['name', 'price', 'type', 'location', 'time', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']]

# Displaying the df
bug_id_merge

In [ ]:
bug_id

## Splitting tables

In [ ]:
# Creating a df for the fish 
bug_df = bug_id_merge.iloc[:,:4]

# Displaying the fish df
bug_df

In [ ]:
# Creating a df for the months fish are available
bug_months_df = bug_id_merge.iloc[:,5:]

# Displaying fish months df
bug_months_df

In [ ]:
# Creating a df for the months fish are available
bug_time_df = bug_id_merge.iloc[:,4:5]

# Displaying fish months df
bug_time_df

## Create database connection

In [ ]:
# Creaate connection
connection_string = f"{username}:{password}@localhost:5432/ACNH_Critterpedia"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
bug_df.to_sql(name='encyclopedia', con=engine, if_exists='append', index=True)
bug_months_df.to_sql(name='months', con=engine, if_exists='append', index=True)
bug_time_df.to_sql(name='times', con=engine, if_exists='append', index=True)